# Installating R on WinPython (version of 2019-08-25)

### Warning: as of 2019-08-25, the R installation is not supposed to support a move of Winpython library

see https://richpauloo.github.io/2018-05-16-Installing-the-R-kernel-in-Jupyter-Lab/

#### This procedure applys for Winpython  (Version of December 2015 and after) 
### 1 - Downloading R binary

In [1]:
import os
import sys
import io

# downloading R may takes a few minutes (80Mo)
try:
    import urllib.request as urllib2  # Python 3
except:
    import urllib2  # Python 2

# specify R binary and (md5, sha1) hash
# R-3.6.1:
r_url = "https://cran.r-project.org/bin/windows/base/old/3.6.1/R-3.6.1-win.exe"
hashes=("f6ca2ecfc66a10a196991b6b6c4e91f6","df4ad3c36e193423ebf2d698186feded15777da1")

# specify target location
# tweak change in recent winpython
tool_base_directory=os.environ["WINPYDIR"]+"\\..\\t\\"
if not os.path.isdir(tool_base_directory):
    tool_base_directory=os.environ["WINPYDIR"]+"\\..\\tools\\"




r_installer = tool_base_directory+os.path.basename(r_url)
os.environ["r_installer"] = r_installer

# Download
g = urllib2.urlopen(r_url) 
with io.open(r_installer, 'wb') as f:
    f.write(g.read())
g.close
g = None

#checking it's there
!dir %r_installer%

URLError: <urlopen error [Errno 11001] getaddrinfo failed>


### 2 - checking and Installing R binary in the right place

In [ ]:
# checking it's the official R
import hashlib
def give_hash(of_file, with_this):
    with io.open(r_installer, 'rb') as f:
        return with_this(f.read()).hexdigest()  
print (" "*12+"MD5"+" "*(32-12-3)+" "+" "*15+"SHA-1"+" "*(40-15-5)+"\n"+"-"*32+" "+"-"*40)
print ("%s %s %s" % (give_hash(r_installer, hashlib.md5) , give_hash(r_installer, hashlib.sha1),r_installer))
if give_hash(r_installer, hashlib.md5) == hashes[0] and give_hash(r_installer, hashlib.sha1) == hashes[1]:
   print("looks good!")
else:
   print("problem ! please check")
   assert give_hash(r_installer, hashlib.md5) == hashes[0]
   assert give_hash(r_installer, hashlib.sha1) == hashes[1]

In [ ]:
# preparing Dos variables
os.environ["R_HOME"] = tool_base_directory+ "R\\" 
os.environ["R_HOMEbin"]=os.environ["R_HOME"] + "bin" 

# for installation we need this
os.environ["tmp_Rbase"]=os.path.join(os.path.split(os.environ["WINPYDIR"])[0]  , 't','R' )  
if 'amd64' in sys.version.lower():
    r_comp ='/COMPONENTS="main,x64,translations'
else:
    r_comp ='/COMPONENTS="main,i386,translations'
os.environ["tmp_R_comp"]=r_comp


In [ ]:
# let's install it, if hashes do match
assert give_hash(r_installer, hashlib.md5) == hashes[0]
assert give_hash(r_installer, hashlib.sha1) == hashes[1]
# If you are "USB life style", or multi-winpython
#   ==> CLICK the OPTION "Don't create a StartMenuFolder' <== (when it will show up)

!start cmd /C %r_installer% /DIR=%tmp_Rbase% %tmp_R_comp%

## During Installation (if you wan't to move the R installation after)

Choose non default option "Yes (customized startup"

then after 3 screens, Select  "Don't create a Start Menu Folder"

Un-select "Create a desktop icon"

Un-select "Save version number in registery"


<img src="https://raw.githubusercontent.com/stonebig/winpython_afterdoc/master/examples/images/r_setup_unclick_shortcut.GIF">

### 3 - create a R_launcher and install irkernel

In [ ]:
import os
import sys
import io
# let's create a R launcher  
r_launcher = r"""
@echo off
call %~dp0env.bat
rscript %*
"""
r_launcher_bat = os.environ["WINPYDIR"]+"\\..\\scripts\\R_launcher.bat"

# let's create a R init script
# in manual command line, you can use repos = c('http://irkernel.github.io/', getOption('repos'))
r_initialization = r"""
install.packages(c('repr', 'IRdisplay', 'evaluate', 'crayon', 'pbdZMQ', 'devtools', 'uuid', 'digest', 'stringr'), repos = c('http://cran.rstudio.com/', 'http://cran.rstudio.com/'))
devtools::install_github('IRkernel/IRkernel')
library('pbdZMQ')
library('repr')
library('IRkernel')
library('IRdisplay')
library('crayon')
library('stringr')
IRkernel::installspec()
"""
# IRkernel::installspec() # install for the current user: 
# IRkernel::installspec(user = FALSE) # install system-wide
r_initialization_r = os.path.normpath(os.environ["WINPYDIR"]+"\\..\\scripts\\R_initialization.r")


for i in [(r_launcher,r_launcher_bat), (r_initialization, r_initialization_r)]:
    with io.open(i[1], 'w', encoding = sys.getdefaultencoding() ) as f:
        for line in i[0].splitlines():
            f.write('%s\n' %  line  )


In [ ]:
#check what we are going to do 
print ("!start cmd /C %WINPYDIR%\\..\\scripts\\R_launcher.bat --no-restore --no-save  " + r_initialization_r)

In [ ]:
# Launch Rkernel setup
os.environ["r_initialization_r"] = r_initialization_r
!start cmd /C %WINPYDIR%\\..\\scripts\\R_launcher.bat --no-restore --no-save  %r_initialization_r%  

### 4- Install a R package via a IPython  Kernel

In [ ]:
%load_ext rpy2.ipython

#vitals: 'dplyr', 'R.utils', 'nycflights13'
# installation takes 2 minutes
%R install.packages(c('dplyr','R.utils', 'nycflights13'), repos='http://cran.rstudio.com/') 

### 5- Small demo via R magic

In [ ]:
!echo %R_HOME%

In [ ]:
%load_ext rpy2.ipython

In [ ]:
# avoid some pandas deprecation warning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
%%R
library('dplyr')
library('nycflights13') 
write.csv(flights, "flights.csv")

In [ ]:
%R head(flights) 

In [ ]:
%R airports %>% mutate(dest = faa) %>% semi_join(flights) %>% head

### 6 - Installing the very best of R pakages (optional, you will start to get a really big directory)


In [ ]:
# essentials: 'tidyr', 'shiny', 'ggplot2', 'caret' , 'nnet'  
# remaining of Hadley Wickahm "stack" (https://github.com/rstudio)
%R install.packages(c('tidyr', 'ggplot2',  'shiny','caret' , 'nnet'), repos='https://cran.rstudio.com/') 
%R install.packages(c('knitr', 'purrr', 'readr', 'readxl'), repos='https://cran.rstudio.com/')
%R install.packages(c('rvest', 'lubridate', 'ggvis', 'readr','base64enc'), repos='https://cran.rstudio.com/')

# TRAINING = online training book http://r4ds.had.co.nz/ (or https://github.com/hadley/r4ds)

### 7 - Relaunch Jupyter Notebook to get a R kernel option
launch a new notebook of "R" type, and type in it:
    
library('dplyr')

library('nycflights13') 

head(flights)

### 9 - To Un-install  / Re-install R (or other trouble-shooting)

- launch  winpython**\t\R\unins000.exe   (was formerly winpython**\tools\R\unins000.exe)

- delete the directory winpython**\t\R (was formerly winpython**\tools\R)

- re-install


In [ ]:
%R  install.packages(c('bindrccp'), repos='http://cran.rstudio.com/') 